In [1]:
%run ../Python_files/util_data_storage_and_load.py

In [2]:
%run ../Python_files/load_dicts.py

In [3]:
%run ../Python_files/util.py

In [4]:
# load logit_route_choice_probability_matrix
P = zload('../temp_files/logit_route_choice_probability_matrix.pkz')

In [5]:
import numpy as np
from numpy.linalg import inv

P = np.matrix(P)

In [6]:
np.size(P, 0), np.size(P, 1)

(56, 140)

In [7]:
import numpy as np
from numpy.linalg import inv

def samp_cov(x):
    """
    x: sample matrix, each column is a link flow vector sample; 24 * K
    K: number of samples
    S: sample covariance matrix
    ----------------
    return: inv(S)
    ----------------
    """
    x = np.matrix(x)
    K = np.size(x, 1)
    x_mean = sum(x[:,k] for k in range(K)) / K
    S = sum(np.dot(x[:,k] - x_mean, np.transpose(x[:,k] - x_mean)) for k in range(K)) / (K - 1)
    return S

In [8]:
# load path-link incidence matrix
A = zload('../temp_files/path-link_incidence_matrix.pkz')

In [9]:
# load link counts data

import json

with open('../temp_files/link_day_minute_Apr_dict_JSON.json', 'r') as json_file:
    link_day_minute_Apr_dict_JSON = json.load(json_file)

In [85]:
#weekend_Apr_list = [1, 7, 8, 14, 15, 21, 22, 28, 29]
weekend_Apr_list = [29]

In [86]:
link_day_minute_Apr_list = []
for link_idx in range(24):
    for day in weekend_Apr_list: 
        for minute_idx in range(120):
            key = 'link_' + str(link_idx) + '_' + str(day)
            link_day_minute_Apr_list.append(link_day_minute_Apr_dict_JSON[key] ['AM_flow_minute'][minute_idx])

In [87]:
len(link_day_minute_Apr_list)

2880

In [88]:
x = np.matrix(link_day_minute_Apr_list)
x = np.matrix.reshape(x, 24, 120)

x = np.nan_to_num(x)
y = np.array(np.transpose(x))
y = y[np.all(y != 0, axis=1)]
x = np.transpose(y)
x = np.matrix(x)

In [89]:
x[:,:2]

matrix([[ 3354.46592907,  2963.39881332],
        [ 2338.51305622,  2720.50331889],
        [ 3902.74728049,  3369.70731368],
        [ 4343.92663555,  4037.38781843],
        [ 3084.90398346,  4924.51618539],
        [ 2876.24011799,  2903.36938298],
        [ 3864.54695307,  3930.93680705],
        [ 3243.28341576,  2796.63199938],
        [ 3199.4504619 ,  3292.98126132],
        [ 3343.32875148,  2494.70113654],
        [ 2868.50066407,  2938.59611649],
        [ 3844.85479029,  2334.84537858],
        [ 2801.53318565,  4170.86075562],
        [ 1754.75959421,  2734.17175647],
        [ 2270.16645106,  2641.25639428],
        [  544.7542192 ,  1550.96837297],
        [ 3310.95406712,  2408.38196337],
        [ 2724.20704972,  3183.58676193],
        [ 2093.66889526,  2697.91510994],
        [ 3494.84595788,  4338.72700904],
        [ 1423.12272724,  2087.91779539],
        [ 1699.32192555,  1878.41541622],
        [ 2930.56802053,  2861.92610347],
        [ 4127.50401686,  4609.490

In [90]:
np.size(x, 0), np.size(x, 1)

(24, 120)

In [91]:
np.size(A,0), np.size(A,1)

(24, 140)

In [92]:
from gurobipy import *

L = 56  # dimension of lam

K = np.size(x, 1)
S = samp_cov(x)
inv_S = inv(S)

A_t = np.transpose(A)
P_t = np.transpose(P)
# PA'
PA_t = np.dot(P, A_t)
# AP_t
AP_t = np.transpose(PA_t)

Q = np.dot(np.dot(PA_t, inv_S), AP_t)
b = sum([np.dot(np.dot(PA_t, inv_S), x[:, k]) for k in range(K)])


model = Model("OD_matrix_estimation")

lam = []
for l in range(L):
    lam.append(model.addVar(name='lam_' + str(l)))

model.update() 

# Set objective: (K/2) lam' * Q * lam - b' * lam
obj = 0
for i in range(L):
    for j in range(L):
        obj += (1.0 /2) * K * lam[i] * Q[i, j] * lam[j]
for l in range(L):
    obj += - b[l] * lam[l]
model.setObjective(obj)

# Add constraint: lam >= 0
for l in range(L):
    model.addConstr(lam[l] >= 0)

model.update() 

model.optimize()

lam_list = []
for v in model.getVars():
    print('%s %g' % (v.varName, v.x))
    lam_list.append(v.x)
print('Obj: %g' % obj.getValue())

Optimize a model with 56 rows, 56 columns and 56 nonzeros
Model has 1596 quadratic objective terms
Coefficient statistics:
  Matrix range    [1e+00, 1e+00]
  Objective range [2e-01, 6e+00]
  Bounds range    [0e+00, 0e+00]
  RHS range       [0e+00, 0e+00]
Presolve removed 56 rows and 0 columns
Presolve time: 0.00s
Presolved: 0 rows, 56 columns, 0 nonzeros
Presolved model has 1596 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 24
 AA' NZ     : 2.760e+02
 Factor NZ  : 3.000e+02
 Factor Ops : 4.900e+03 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0  -3.05753460e+06  0.00000000e+00  9.77e+03 1.43e+02  9.31e+05     0s
   1   3.10804076e+06 -3.96955991e+06  1.47e+02 1.14e-13  1.40e+05     0s
   2   3.91656933e+05 -7.50496936e+05  1.47e-04 5.68e-14  2.04e+04     0s
   3  -6.58490580e+03 -1.66844799e+05  1.47e-10 2.84e-14  2.8

In [93]:
# write estimation result to file
n = 8  # number of nodes
with open('../temp_files/OD_demand_matrix_Apr_29_weekend.txt', 'w') as the_file:
    idx = 0
    for i in range(n + 1)[1:]:
        for j in range(n + 1)[1:]:
            if i != j: 
                the_file.write("%d,%d,%f\n" %(i, j, lam_list[idx]))
                idx += 1

In [94]:
lam_list

[5.599142076664907e-10,
 1462.7038846919995,
 6.190252549938277e-10,
 226.26649018602458,
 440.68371436199254,
 4130.73284774021,
 1.8684290223928643e-07,
 1733.8319870217338,
 4.327638311952276e-09,
 3339.6001719347555,
 4.327645320159098e-09,
 4.327641136046096e-09,
 4.327639095853317e-09,
 4.352457120088865e-09,
 1026.879507876131,
 614.9703512154442,
 3.0044932333914945e-07,
 253.28673199377408,
 678.9684719779656,
 1.9546526168669892e-05,
 0.001107927508293416,
 8.423763538763706e-06,
 2920.4779395354203,
 6.222366366778314e-07,
 2966.0150622976735,
 639.8015482526905,
 55.307354578236,
 2361.1052202019205,
 2100.4171845027304,
 753.8809573235212,
 5.8502822242587e-07,
 1473.3020967372543,
 1202.2787163736864,
 2.197046729565075e-07,
 0.0009637317835601011,
 2674.3602723205045,
 807.5377526625059,
 5.468711042561428e-07,
 1449.7038860891996,
 0.00038000989279964037,
 0.006179130146833023,
 0.003471631707992029,
 5.919023293271644e-07,
 5.919299051661292e-07,
 2.710183452005329e-07